In [2]:
import os
import re
import json
from pathlib import Path
import xml.etree.ElementTree as ET
import html

In [ ]:
def remove_space(text):
  text = html.unescape(text)
  text = re.sub("\u00a0", " ", text)
  text = re.sub(r"[ \t]+", " ", text)
  text = re.sub(r"\n{2,}", "\n", text)
  text = text.strip()
  return text

def process_조문(j:ET.Element):
  lines = []

  no = remove_space(j.findtext("조문번호", ""))
  title = remove_space(j.findtext("조문제목", ""))
  body = remove_space(j.findtext("조문내용", ""))

  header = f"제{no}조" if no else ""
  header += f"({title})" if title else ""

  if header:
    lines.append(header)

  if body:
    lines.append(body)


  항_list = j.findall("./항")
  if 항_list:
    for 항 in 항_list:
      항_text = remove_space(항.findtext("항내용", ""))
      항_no = remove_space(항.findtext("항번호", ""))

      if 항_text:
        lines.append(항_text.strip())

      호_list = 항.findall("./호")
      if 호_list:
        for 호 in 호_list:
          호_text = remove_space(호.findtext("호내용", ""))
          호_no = remove_space(호.findtext("호번호", ""))

          if 호_text or 호_no:
            lines.append(f"- {호_no} {호_text}".strip())

          목_list = 호.findall("./목")
          if 목_list:
            for 목 in 목_list:
              목_text = remove_space(목.findtext("목내용", ""))
              목_no = remove_space(목.findtext("목번호", ""))
              if 목_text or 목_no:
                lines.append(f"-- {목_no} {목_text}".strip())

  else:
    호_list = j.findall(".//호")
    if 호_list:
      for 호 in 호_list:
        호_text = remove_space(호.findtext("호내용", ""))
        호_no = remove_space(호.findtext("호번호", ""))
        if 호_no or 호_text:
          lines.append(f"- {호_no} {호_text}".strip())

        목_list = 호.findall("./목")
        if 목_list:
          for 목 in 목_list:
            목_text = remove_space(목.findtext("목내용", ""))
            목_no = remove_space(목.findtext("목번호", ""))
            if 목_text or 목_no:
              lines.append(f"-- {목_no} {목_text}".strip())


  return "\n".join([x for x in lines if x])


In [ ]:
root = ET.parse("./drive/MyDrive/rag/load_data/법인세법.xml").getroot()

name = remove_space(root.findtext(".//기본정보/법령명_한글", ""))
id = remove_space(root.findtext(".//기본정보/법령ID", ""))
공포일자 = remove_space(root.findtext(".//기본정보/공포일자", ""))
시행일자 = remove_space(root.findtext(".//기본정보/시행일자", ""))

base_path = "./drive/MyDrive/rag/preprocessing_rag/"

with open(f'{base_path}/{name}.jsonl', 'w', encoding="utf-8") as f:
  for j in root.findall(".//조문단위"):
    if j.findtext("조문여부", "") == "조문":
      no = remove_space(j.findtext("조문번호", ""))
      title = remove_space(j.findtext("조문제목", ""))
      text = process_조문(j)

      if not text:
        continue

      j_data = {
        "law_name": name,
        "law_id": id,
        "공포일자": 공포일자,
        "시행일자": 시행일자,
        "조문번호": no,
        "조문제목": title,
        "조문내용": text,
        "source": "법령정보센터"
      }

      f.write(json.dumps(j_data, ensure_ascii=False) + "\n")


In [ ]:
def save_jsonal(path):
  root = ET.parse(path).getroot()

  name = remove_space(root.findtext(".//기본정보/법령명_한글", ""))
  id = remove_space(root.findtext(".//기본정보/법령ID", ""))
  공포일자 = remove_space(root.findtext(".//기본정보/공포일자", ""))
  시행일자 = remove_space(root.findtext(".//기본정보/시행일자", ""))

  base_path = "./drive/MyDrive/rag/preprocessing_rag/"

  with open(f'{base_path}/{name}.jsonl', 'w', encoding="utf-8") as f:
    for j in root.findall(".//조문단위"):
      if j.findtext("조문여부", "") == "조문":
        no = remove_space(j.findtext("조문번호", ""))
        title = remove_space(j.findtext("조문제목", ""))
        text = process_조문(j)

        if not text:
          continue

        j_data = {
          "law_name": name,
          "law_id": id,
          "공포일자": 공포일자,
          "시행일자": 시행일자,
          "조문번호": no,
          "조문제목": title,
          "조문내용": text,
          "source": "법령정보센터"
        }

        f.write(json.dumps(j_data, ensure_ascii=False) + "\n")

    return name, id, 공포일자, 시행일자


In [ ]:
save_jsonal("./drive/MyDrive/rag/load_data/법인세법.xml")

('법인세법', '001563', '20251001', '20251001')

In [3]:
class preprocessing():
  def __init__(self, law_name):
    self.path = "./drive/MyDrive/rag/load_data/"+law_name+".xml"
    self.root = ET.parse(self.path).getroot()

  def remove_space(self, text):
    text = html.unescape(text)
    text = re.sub("\u00a0", " ", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{2,}", "\n", text)
    text = text.strip()
    return text

  def remove_duplicate(self, body, no):
    pattern = rf"""
    ^제\s*{re.escape(no)}\s*조
    (?:\s*의\s\d+)?
    \s*\([^)]*\)
    \s*
    """

    if not body:
      return body

    body = re.sub(pattern, "", body, count=1, flags = re.VERBOSE).strip()
    return body


  def process_조문(self, j:ET.Element)->list[dict]: # {level, 항번호, 호번호, 목번호} 등 포함
    lines = []

    no = self.remove_space(j.findtext("조문번호", ""))
    sub_no = self.remove_space(j.findtext("조문가지번호", ""))
    sub_no = sub_no if sub_no else None
    title = self.remove_space(j.findtext("조문제목", ""))
    body = self.remove_space(j.findtext("조문내용", ""))

    header = f"제{no}조" if no else ""
    header += f"의{sub_no}" if sub_no else ""
    header += f"({title})" if title else ""

    if body:
      body = self.remove_duplicate(body, no)
      lines.append({
          "level": "조문내용", "text": f"{header} {body}".strip() if header else body, "항번호": None, "호번호": None, "목번호": None
      })

    항_list = j.findall("./항")
    if 항_list:
      for 항 in 항_list:
        항_text = self.remove_space(항.findtext("항내용", ""))
        항_no = self.remove_space(항.findtext("항번호", ""))

        if 항_text:
          lines.append({
              "level": "항",
              "text": 항_text.strip(),
              "항번호": 항_no,
              "호번호": None,
              "목번호": None
          })

        호_list = 항.findall("./호")
        if 호_list:
          for 호 in 호_list:
            호_text = self.remove_space(호.findtext("호내용", ""))
            호_no = self.remove_space(호.findtext("호번호", ""))

            if 호_text:
              lines.append({
                  "level": "호",
                  "text": 호_text.strip(),
                  "항번호": 항_no or None,
                  "호번호": 호_no or None,
                  "목번호": None
              })

            목_list = 호.findall("./목")
            if 목_list:
              for 목 in 목_list:
                목_text = self.remove_space(목.findtext("목내용", ""))
                목_no = self.remove_space(목.findtext("목번호", ""))
                if 목_text:
                  lines.append({
                      "level": "목",
                      "text": 목_text.strip(),
                      "항번호": 항_no or None,
                      "호번호": 호_no or None,
                      "목번호": 목_no or None
                  })

    else:
      호_list = j.findall(".//호")
      if 호_list:
        for 호 in 호_list:
          호_text = self.remove_space(호.findtext("호내용", ""))
          호_no = self.remove_space(호.findtext("호번호", ""))
          if 호_no:
            lines.append({
                "level": "호",
                "text": 호_text.strip(),
                "항번호": None,
                "호번호": 호_no or None,
                "목번호": None
            })

          목_list = 호.findall("./목")
          if 목_list:
            for 목 in 목_list:
              목_text = self.remove_space(목.findtext("목내용", ""))
              목_no = self.remove_space(목.findtext("목번호", ""))
              if 목_text:
                lines.append({
                    "level": "목",
                    "text": 목_text.strip(),
                    "항번호": None,
                    "호번호": 호_no or None,
                    "목번호": 목_no or None
                })


    return lines


  def save_jsonal(self):
    name = self.remove_space(self.root.findtext(".//기본정보/법령명_한글", ""))
    id = self.remove_space(self.root.findtext(".//기본정보/법령ID", ""))
    공포일자 = self.remove_space(self.root.findtext(".//기본정보/공포일자", ""))
    시행일자 = self.remove_space(self.root.findtext(".//기본정보/시행일자", ""))

    base_path = "./drive/MyDrive/rag/preprocessing_rag/"
    os.makedirs(base_path, exist_ok=True)

    with open(f'{base_path}/{name}.jsonl', 'w', encoding="utf-8") as f:
      for j in self.root.findall(".//조문단위"):
        if j.findtext("조문여부", "") == "조문":
          no = self.remove_space(j.findtext("조문번호", ""))
          sub_no = self.remove_space(j.findtext("조문가지번호", ""))
          sub_no = sub_no if sub_no else None
          title = self.remove_space(j.findtext("조문제목", ""))
          text = self.process_조문(j)

          if not text:
            continue

          for i, t in enumerate(text, 1):
            j_data = {
              "law_name": name,
              "law_id": id,
              "공포일자": 공포일자,
              "시행일자": 시행일자,
              "조문번호": no,
              "조문가지번호": sub_no,
              "조문제목": title,
              "level": t["level"],
              "항번호": t["항번호"],
              "호번호": t["호번호"],
              "목번호": t["목번호"],
              "chunk_index": i,
              "chunk_text": t["text"],
              "source": "법령정보센터"
            }

            f.write(json.dumps(j_data, ensure_ascii=False) + "\n")

      return name, id, 공포일자, 시행일자


In [4]:
법인세법 = preprocessing("법인세법")


In [5]:
법인세법.save_jsonal()

('법인세법', '001563', '20251001', '20251001')

In [6]:
법인세법_시행규칙 = preprocessing("법인세법_시행규칙")
법인세법_시행규칙.save_jsonal()

('법인세법 시행규칙', '007229', '20260102', '20260102')

In [7]:
법인세법_시행령 = preprocessing("법인세법_시행령")
법인세법_시행령.save_jsonal()

('법인세법 시행령', '003608', '20251230', '20260102')

In [8]:
부가가치세법 = preprocessing("부가가치세법")
부가가치세법.save_jsonal()

('부가가치세법', '001571', '20251001', '20260102')

In [9]:
부가가치세법_시행령 = preprocessing("부가가치세법_시행령")
부가가치세법_시행령.save_jsonal()

('부가가치세법 시행령', '003666', '20251230', '20260102')

In [10]:
부가가치세법_시행규칙 = preprocessing("부가가치세법_시행규칙")
부가가치세법_시행규칙.save_jsonal()

('부가가치세법 시행규칙', '007289', '20260102', '20260102')

In [11]:
소득세법 = preprocessing("소득세법")

In [12]:
소득세법.save_jsonal()

('소득세법', '001565', '20251001', '20251001')

In [13]:
소득세법_시행령 = preprocessing("소득세법_시행령")
소득세법_시행령.save_jsonal()

('소득세법 시행령', '003956', '20251230', '20260102')

In [14]:
소득세법_시행규칙 = preprocessing("소득세법_시행규칙")
소득세법_시행규칙.save_jsonal()

('소득세법 시행규칙', '007507', '20260102', '20260102')